# Import Librerie

In [46]:
from pprint import pprint

# Struttura dei dati StatsBomb open-data

In questa sezione esploro la folder `open-data/data` e mostro la struttura dei JSON.


In [47]:
from pathlib import Path
import json

DATA_ROOT = Path("/Users/lorenzoguercio/Documents/Projects/sport_data/open-data/data")

# Stampa una vista sintetica della struttura della cartella dati.
def list_tree(root: Path, max_files_per_dir: int = 5):
    print(f"Root: {root}")
    if not root.exists():
        raise FileNotFoundError(f"DATA_ROOT non esiste: {root}")
    for item in sorted(root.iterdir()):
        if item.is_dir():
            files = [p.name for p in sorted(item.iterdir()) if p.is_file()]
            print(f"- {item.name}/ ({len(files)} files)")
            if files:
                preview = files[:max_files_per_dir]
                print(f"  es: {', '.join(preview)}")
        else:
            print(f"- {item.name}")

# Legge un JSON e ne riassume il tipo e le chiavi principali.
def describe_json(path: Path, max_keys: int = 20):
    if not path.exists():
        raise FileNotFoundError(f"File non trovato: {path}")
    try:
        data = json.loads(path.read_text())
    except json.JSONDecodeError as e:
        raise ValueError(f"JSON non valido in {path}: {e}") from e

    if isinstance(data, list):
        print(f"{path.name}: list (len={len(data)})")
        if data and isinstance(data[0], dict):
            keys = list(data[0].keys())
            print(f"  keys[0]: {keys[:max_keys]}")
    elif isinstance(data, dict):
        keys = list(data.keys())
        print(f"{path.name}: dict")
        print(f"  keys: {keys[:max_keys]}")
    else:
        print(f"{path.name}: {type(data).__name__}")

# Prende il primo JSON direttamente dentro una cartella.
def first_json_file(folder: Path):
    if not folder.exists():
        raise FileNotFoundError(f"Cartella non trovata: {folder}")
    files = [p for p in sorted(folder.iterdir()) if p.is_file() and p.suffix == ".json"]
    if not files:
        raise FileNotFoundError(f"Nessun JSON in: {folder}")
    return files[0]

# Gestisce matches/: scandisce le subfolder e mostra un sample per ognuna.
def describe_matches(root: Path, max_folders: int = 5):
    if not root.exists():
        raise FileNotFoundError(f"Cartella non trovata: {root}")
    folders = [p for p in sorted(root.iterdir()) if p.is_dir()]
    print(f"matches/: {len(folders)} folders")
    for folder in folders[:max_folders]:
        files = [p for p in sorted(folder.iterdir()) if p.is_file() and p.suffix == ".json"]
        print(f"- {folder.name}/ ({len(files)} files)")
        if files:
            sample = files[0]
            print(f"  sample: {sample.name}")
            describe_json(sample)

list_tree(DATA_ROOT)

# Esempi di struttura per ciascuna sottofolder

describe_json(DATA_ROOT / "competitions.json")

describe_matches(DATA_ROOT / "matches")

for sub in ["events", "lineups", "three-sixty"]:
    sample = first_json_file(DATA_ROOT / sub)
    print(f"\nSample from {sub}: {sample.name}")
    describe_json(sample)


Root: /Users/lorenzoguercio/Documents/Projects/sport_data/open-data/data
- .DS_Store
- competitions.json
- events/ (3464 files)
  es: 15946.json, 15956.json, 15973.json, 15978.json, 15986.json
- lineups/ (3464 files)
  es: 15946.json, 15956.json, 15973.json, 15978.json, 15986.json
- matches/ (0 files)
- three-sixty/ (326 files)
  es: 3788741.json, 3788742.json, 3788743.json, 3788744.json, 3788745.json
competitions.json: list (len=75)
  keys[0]: ['competition_id', 'season_id', 'country_name', 'competition_name', 'competition_gender', 'competition_youth', 'competition_international', 'season_name', 'match_updated', 'match_updated_360', 'match_available_360', 'match_available']
matches/: 21 folders
- 11/ (18 files)
  sample: 1.json
1.json: list (len=36)
  keys[0]: ['match_id', 'match_date', 'kick_off', 'competition', 'season', 'home_team', 'away_team', 'home_score', 'away_score', 'match_status', 'match_status_360', 'last_updated', 'last_updated_360', 'metadata', 'match_week', 'competition

# EDA mirata per xG (StatsBomb open-data + 360)

**Obiettivo**: stimare `P(goal | shot context + 360)`

**Output atteso**: dataset *one row per shot* + baseline model + checklist next steps.

**Scope**: *open play only* (escludo corner, punizioni, throw-in, penalty).


In [53]:
from pathlib import Path
import json
import math
import random
import pandas as pd

random.seed(42)

DATA_ROOT = Path("/Users/lorenzoguercio/Documents/Projects/sport_data/open-data/data")
EVENTS_DIR = DATA_ROOT / "events"
THREESIXTY_DIR = DATA_ROOT / "three-sixty"

OPEN_PLAY_PATTERNS = {"Regular Play", "From Counter", "From Keeper", "Other"}

def distance_to_goal(x, y, goal_x=120, goal_y=40):
    return math.hypot(goal_x - x, goal_y - y)

def shot_angle(x, y, goal_x=120, goal_y=40, goal_width=7.32):
    half_width = goal_width / 2
    left_post = (goal_x, goal_y - half_width)
    right_post = (goal_x, goal_y + half_width)
    a = math.hypot(left_post[0] - x, left_post[1] - y)
    b = math.hypot(right_post[0] - x, right_post[1] - y)
    c = goal_width
    # Legge del coseno con clipping numerico
    cos_angle = (a*a + b*b - c*c) / (2*a*b) if a > 0 and b > 0 else 1.0
    cos_angle = max(-1.0, min(1.0, cos_angle))
    return math.acos(cos_angle)

def flip_coordinates(x, y, pitch_length=120, pitch_width=80):
    return pitch_length - x, pitch_width - y

def is_open_play(event):
    play_pattern = event.get("play_pattern", {})
    return play_pattern.get("name") in OPEN_PLAY_PATTERNS


## 1) Caricamento dati e inventory

### 1.1 Load di una singola partita di test


In [54]:
# Scegli un match che abbia sia events che 360
match_ids = sorted({p.stem for p in EVENTS_DIR.glob('*.json')} & {p.stem for p in THREESIXTY_DIR.glob('*.json')})
MATCH_ID = match_ids[0] if match_ids else None
print(f"Test match_id: {MATCH_ID}")

events_path = EVENTS_DIR / f"{MATCH_ID}.json"
three_sixty_path = THREESIXTY_DIR / f"{MATCH_ID}.json"

events = json.loads(events_path.read_text())
three_sixty = json.loads(three_sixty_path.read_text())

print(f"N Events: {len(events)}")
print(f"N 360 Frames: {len(three_sixty)}")
print(f"Events Keys (sample): {list(events[0].keys())[:30]}")
print(f"360 Keys (sample): {list(three_sixty[0].keys())[:15]}")

def list_keys_by_level(obj, max_level=4):
    levels = {1: set(), 2: set(), 3: set(), 4: set()}

    def walk(o, level):
        if level > max_level:
            return
        if isinstance(o, dict):
            if level in levels:
                levels[level].update(o.keys())
            for v in o.values():
                walk(v, level + 1)
        elif isinstance(o, list):
            for item in o:
                walk(item, level)

    walk(obj, 1)
    for lvl in range(1, max_level + 1):
        keys = sorted(levels[lvl])
        print(f"LIVELLO {lvl}: {keys}")

print("\nKeys events per livello (tutti gli eventi):")
list_keys_by_level(events, max_level=4)

print("\nKeys 360 per livello (tutti i frame):")
list_keys_by_level(three_sixty, max_level=3)


Test match_id: 3788741
N Events: 3803
N 360 Frames: 3370
Events Keys (sample): ['id', 'index', 'period', 'timestamp', 'minute', 'second', 'type', 'possession', 'possession_team', 'play_pattern', 'team', 'duration', 'tactics']
360 Keys (sample): ['event_uuid', 'visible_area', 'freeze_frame']

Keys events per livello (tutti gli eventi):
LIVELLO 1: ['ball_receipt', 'ball_recovery', 'block', 'carry', 'clearance', 'counterpress', 'dribble', 'duel', 'duration', 'foul_committed', 'foul_won', 'goalkeeper', 'id', 'index', 'interception', 'location', 'minute', 'miscontrol', 'off_camera', 'out', 'pass', 'period', 'play_pattern', 'player', 'position', 'possession', 'possession_team', 'related_events', 'second', 'shot', 'substitution', 'tactics', 'team', 'timestamp', 'type', 'under_pressure']
LIVELLO 2: ['advantage', 'aerial_won', 'angle', 'assisted_shot_id', 'body_part', 'card', 'cross', 'defensive', 'deflected', 'deflection', 'end_location', 'first_time', 'formation', 'freeze_frame', 'goal_assist

In [55]:
# Chiave di collegamento events ↔ 360
print("Event id key: id")
print("360 key: event_uuid")

# Esempio di match tra i due
example_event_id = events[101].get('id')
matching_frames = [f for f in three_sixty if f.get('event_uuid') == example_event_id]
# pprint(matching_frames)
print(f"Esempio event id: {example_event_id}")
print(f"Frames 360 che matchano: {len(matching_frames)}")


Event id key: id
360 key: event_uuid
Esempio event id: 30b3496b-e595-45c3-b28a-fed88dff7fc1
Frames 360 che matchano: 1


### 1.2 Data coverage
Decisione: usare 360 solo quando presente oppure filtrare a sole partite con 360.


In [56]:
event_ids = {p.stem for p in EVENTS_DIR.glob('*.json')}
three_sixty_ids = {p.stem for p in THREESIXTY_DIR.glob('*.json')}

matches_with_360 = sorted(event_ids & three_sixty_ids)
print(f"matches con 360: {len(matches_with_360)} / {len(event_ids)}")

total_events = 0
total_frames = 0
total_events_with_360 = 0
total_shots = 0
total_shots_with_360 = 0

invalid_360 = []

for mid in matches_with_360:
    ev = json.loads((EVENTS_DIR / f"{mid}.json").read_text())
    th_path = THREESIXTY_DIR / f"{mid}.json"
    try:
        th_text = th_path.read_text()
        th = json.loads(th_text)
    except json.JSONDecodeError as e:
        invalid_360.append({
            "match_id": mid,
            "path": str(th_path),
            "error": str(e),
            "size": th_path.stat().st_size if th_path.exists() else None,
        })
        continue

    total_events += len(ev)
    total_frames += len(th)
    th_event_ids = {f.get('event_uuid') for f in th}
    total_events_with_360 += sum(1 for e in ev if e.get('id') in th_event_ids)
    shots = [e for e in ev if e.get('type', {}).get('name') == 'Shot']
    total_shots += len(shots)
    total_shots_with_360 += sum(1 for s in shots if s.get('id') in th_event_ids)

event_coverage = total_events_with_360 / total_events if total_events else 0
shot_coverage = total_shots_with_360 / total_shots if total_shots else 0

print(f"% eventi con 360 (solo match con 360): {event_coverage:.2%}")
print(f"% shot con 360 (solo match con 360): {shot_coverage:.2%}")

if invalid_360:
    print(f"File 360 non validi: {len(invalid_360)}")
    for item in invalid_360[:10]:
        print(f"- {item['match_id']}: {item['path']} | {item['error']} | size={item['size']}")


matches con 360: 326 / 3464
% eventi con 360 (solo match con 360): 87.10%
% shot con 360 (solo match con 360): 96.49%
File 360 non validi: 3
- 3835338: /Users/lorenzoguercio/Documents/Projects/sport_data/open-data/data/three-sixty/3835338.json | Expecting value: line 181321 column 20 (char 5193728) | size=5959607
- 3835342: /Users/lorenzoguercio/Documents/Projects/sport_data/open-data/data/three-sixty/3835342.json | Expecting ',' delimiter: line 171856 column 109 (char 4882432) | size=5942292
- 3845506: /Users/lorenzoguercio/Documents/Projects/sport_data/open-data/data/three-sixty/3845506.json | Expecting ',' delimiter: line 92794 column 3 (char 2637824) | size=6260494


In [57]:
# Lista completa dei play_pattern presenti (match con 360)
play_patterns = set()
for mid in matches_with_360:
    ev = json.loads((EVENTS_DIR / f"{mid}.json").read_text())
    for e in ev:
        name = e.get('play_pattern', {}).get('name')
        if name:
            play_patterns.add(name)

print(sorted(play_patterns))


['From Corner', 'From Counter', 'From Free Kick', 'From Goal Kick', 'From Keeper', 'From Kick Off', 'From Throw In', 'Other', 'Regular Play']


### 1.3 Validazione join events ↔ 360
Output: tabella di join quality (match rate, missing, duplicates).


In [58]:
ev_ids = [e.get('id') for e in events]
th_ids = [f.get('event_uuid') for f in three_sixty]

ev_set = set(ev_ids)
th_set = set(th_ids)

match_rate = len(ev_set & th_set) / len(ev_set) if ev_set else 0
missing_in_360 = len(ev_set - th_set)
missing_in_events = len(th_set - ev_set)
duplicates = len(th_ids) - len(th_set)

join_quality = pd.DataFrame([
    {
        'match_id': MATCH_ID,
        'match_rate': match_rate,
        'events_missing_360': missing_in_360,
        'frames_missing_event': missing_in_events,
        'duplicate_360_event_uuid': duplicates,
    }
])
join_quality


,match_id,match_rate,events_missing_360,frames_missing_event,duplicate_360_event_uuid
0,3788741,0.886143,433,0,0


## 2) Definizione del target e shot universe (open play)

In questa sezione considero solo **open play**. Penalty e free kick vengono esclusi.


In [59]:
# 2.1 Estrazione Shot (open play): singolo match + full dataset
# Singola partita di test (già caricata in 1.1)
shots_match = [e for e in events if e.get('type', {}).get('name') == 'Shot']

def shot_outcome_is_goal(shot_event):
    outcome = shot_event.get('shot', {}).get('outcome', {})
    return outcome.get('name') == 'Goal'

def shot_type_name(shot_event):
    return shot_event.get('shot', {}).get('type', {}).get('name')

open_play_shots_match = [s for s in shots_match if is_open_play(s)]
open_play_shots_match = [s for s in open_play_shots_match if shot_type_name(s) not in {"Penalty", "Free Kick"}]

print(f"shots totali (singolo match): {len(shots_match)}")
print(f"shots open play (singolo match): {len(open_play_shots_match)}")
y_match = [1 if shot_outcome_is_goal(s) else 0 for s in open_play_shots_match]
print(f"goal rate (open play, singolo match): {sum(y_match)/len(y_match) if y_match else 0:.2%}")


shots totali (singolo match): 27
shots open play (singolo match): 6
goal rate (open play, singolo match): 16.67%


In [60]:
# Full dataset: tutti i match con 360
# Riutilizzo matches_with_360 calcolato nella sezione 1.2
all_events = []
for mid in matches_with_360:
    ev_path = EVENTS_DIR / f"{mid}.json"
    all_events.extend(json.loads(ev_path.read_text()))

shots = [e for e in all_events if e.get('type', {}).get('name') == 'Shot']
open_play_shots = [s for s in shots if is_open_play(s)]
open_play_shots = [s for s in open_play_shots if shot_type_name(s) not in {"Penalty", "Free Kick"}]

print(f"shots totali (match con 360): {len(shots)}")
print(f"shots open play (match con 360): {len(open_play_shots)}")
y = [1 if shot_outcome_is_goal(s) else 0 for s in open_play_shots]
print(f"goal rate (open play, match con 360): {sum(y)/len(y) if y else 0:.2%}")

shots totali (match con 360): 8512
shots open play (match con 360): 3290
goal rate (open play, match con 360): 10.36%


In [61]:
# 2.2 Distribuzione del target (open play)
# Aggrego su tutti i match con 360
def rate_by(key_fn):
    buckets = {}
    for s in open_play_shots:
        key = key_fn(s) or 'Unknown'
        if key not in buckets:
            buckets[key] = {"n": 0, "g": 0}
        buckets[key]["n"] += 1
        buckets[key]["g"] += 1 if shot_outcome_is_goal(s) else 0
    rows = []
    for k, v in buckets.items():
        rows.append({"group": k, "shots": v["n"], "goal_rate": v["g"]/v["n"] if v["n"] else 0})
    return pd.DataFrame(rows).sort_values(by='shots', ascending=False)

by_shot_type = rate_by(lambda s: s.get('shot', {}).get('type', {}).get('name'))
by_body_part = rate_by(lambda s: s.get('shot', {}).get('body_part', {}).get('name'))
by_play_pattern = rate_by(lambda s: s.get('play_pattern', {}).get('name'))

print("Goal rate per shot.type (open play):")
display(by_shot_type.head(10))
print("Goal rate per body_part (open play):")
display(by_body_part.head(10))
print("Goal rate per play_pattern (open play):")
display(by_play_pattern.head(10))


Goal rate per shot.type (open play):


,group,shots,goal_rate
0,Open Play,3290,0.103647


Goal rate per body_part (open play):


,group,shots,goal_rate
1,Right Foot,1880,0.095745
0,Left Foot,1079,0.102873
2,Head,319,0.150470
3,Other,12,0.166667


Goal rate per play_pattern (open play):


,group,shots,goal_rate
0,Regular Play,2750,0.097091
1,From Counter,370,0.132432
3,From Keeper,143,0.125874
2,Other,27,0.259259


**Decisioni (open play):**
- I rigori non entrano nel dataset principale (trattati separatamente).
- I free kick non entrano nel dataset principale.
- Modello unico per open play; `shot.type` resta come feature.
